In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import ast
import wandb

api = wandb.Api()
entity = "disco-conv-splade"   # from your code
project = "disco-conv-splade_evaluation"

runs = api.runs(f"{entity}/{project}")

run_list = []
for run in runs:
    if "ckpt" in ast.literal_eval(run.config["config"])['out_dir']:
        run_list.append(run)
df_list = []

for run in run_list:
    # convert summary (dict-like) to DataFrame with one row
    tmp_df = pd.DataFrame([dict(run.summary)])
    
    # extract run_name from config
    config_dict = ast.literal_eval(run.config["config"])
    run_name = int(config_dict['checkpoint_dir'].split('-')[-1])
    
    tmp_df['run_name'] = run_name
    df_list.append(tmp_df)

# concatenate all rows into a single DataFrame
runs_df = pd.concat(df_list, ignore_index=True)

runs_df.head()
# Filter metric columns
metric_cols = [col for col in runs_df.columns if col.startswith("TOPIOCQA/")]

# Sort by run_name
runs_df = runs_df.sort_values(by="run_name")
x = runs_df['run_name']

plt.figure(figsize=(12, 7))

# Plot each metric
for col in metric_cols:
    plt.plot(x, runs_df[col], marker='o', label=col.split('/')[-1])  # show short name in legend

# Labels and title
plt.xlabel('Run Name')
plt.ylabel('Metric Value')
plt.title('TOPIOCQA Metrics per Run')
plt.xticks(x, rotation=45)  # rotate x labels for readability
plt.grid(True, linestyle='--', alpha=0.5)
plt.legend(loc='upper left', bbox_to_anchor=(1,1))  # place legend outside
plt.tight_layout()

plt.show()